In [1]:
import ipywidgets as widgets
import numpy as np
import pandas as pd

import core.config as config
from core.query_manager import QueryManager
from fashion_recommendation import FashionRecommender

from sklearn.linear_model.stochastic_gradient import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import Perceptron
from sklearn.naive_bayes import MultinomialNB

C:\Users\User\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.linear_model.stochastic_gradient module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.linear_model. Anything that cannot be imported from sklearn.linear_model is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
user = "B2F7F3F5-E463-4B4C-8263-D52B16087E67"
# user = 'A9E23C0D-9EB7-4BB8-9C21-B74500DD82D2'
numberResults = 50
recalc = False
searchTerm = 'hawaiian shirt'
threshold = 1.
db_manager = QueryManager(user=user)

#### Recommender functions

In [3]:
def execute_recommendation(recommender):
    # Get search information
    recommender.searchID, recommender.searchTerm, recommender.numberResults = recommender.get_search_details()
    # Get all product ranking attributes
    products_df = recommender.getAttributes()

    # Preprocess ranking attributes
    # products_df = recommender.attributePreprocessing()
    products_df = recommender.orderingPreprocessing(products_df)

    #
    # Score calculation
    #
    # Calculate recommendation score according to text
    products_df = recommender.textBasedScore(products_df)
    # Calculate recommendation score according to ordering
    products_df = recommender.orderingBasedScore(products_df)
    # Calculate recommendation score according to user's feedback
    products_df = recommender.feedbackBasedScore(products_df)
    #
    # 
    products_df = recommender.recommendationScore(products_df, text_mult=1, ordering_mult=.5)
    #
    # Register recommendation to the "Result" table if system's state!=recalc, otherwise update 
    # existing recommendation
    recommender.registerRecommendation(products_df)

    evalRecommendation(recommender, products_df)
    
    

#### Helper functions

In [4]:
def registerSearch(searchTerm, numberResults, user):
    '''CL action'''
    if numberResults==0:
        numberResults = 10

    params = {'table': 'Search', 
              'Criteria': searchTerm, 
              'NumberOfProductsToReturn': numberResults}
    search_df = db_manager.runInsertQuery(params, get_identity=True)

    searchID = search_df['Oid'].values[0]
    return searchID
        
def registerInteraction(recommender, oid, isIrrelevant, grade):
    '''CL action: Updates the recommendation as a new record in Results table'''
    uniq_params = {'table': 'Result', 'Product': oid, 'Search': recommender.searchID, 'GradeByUser': None}    
    params = {'table': 'Result', 
            'Search': recommender.searchID,
            'IsIrrelevant': isIrrelevant,
            'GradeByUser': grade,
            'Product': oid}
    recommender.db_manager.runCriteriaUpdateQuery(uniq_params=uniq_params, params=params)
        
def do_interaction(recommender):
    print('Commiting interaction')
    for (p,c),(p,s) in zip(checkbox_dict.items(), slider_dict.items()):
        registerInteraction(recommender, p, c.value, s.value)        

def evalRecommendation(recommender, recommendation_df):
    where = ' OR '.join(['Oid=%s' % oid for oid in recommendation_df['Oid'].values])
    query = ''' 
            SELECT Oid, Metadata, ProductTitle, Description, ImageSource, Image  FROM %s.dbo.Product
            WHERE %s
            ''' % (config.DB_NAME, where)
    products_df = recommender.db_manager.runSimpleQuery(query, get_identity=True)
    products_df['combined_columns'] = products_df['Metadata'].astype(str) \
                                    + products_df['ProductTitle'].astype(str) \
                                    + products_df['Description'].astype(str)   
    cnt = 0
    for _, row in recommendation_df.iterrows():
        if cnt > numberResults:
            break
        cnt += 1
        oid = row['Oid']
        text_score = row['text_score']
        ordering_score = row['ordering_score']        
        feedback_score = row['feedback_score'] if 'feedback_score' in recommendation_df.columns else np.nan
        final_score = row['final_score']        
        
        # Fetch image
        imageBlob = products_df.loc[products_df['Oid']==oid, 'Image'].values[0]
        imgURL = products_df.loc[products_df['Oid']==oid, 'ImageSource'].values[0].replace('\\', '')
        image = recommender.helper.convertBlobToImage(imageBlob)
        if image is None or image.shape[2] != 3:
            print(imgURL)
            image = recommender.helper.getWebImage(imgURL)
            
        cv_imshow(image, str(oid))
        # relevance widget
        checkbox_dict[oid] = widgets.Checkbox(False, description='IsIrrelevant')
        # rating widget
        slider_dict[oid] = widgets.IntSlider(value=-1, min=-1, max=5, step=1, description='Rating:',
                                            readout=True, readout_format='d')
        all_widgets = widgets.VBox([slider_dict[oid], checkbox_dict[oid]])
        display(all_widgets)
        
        text = products_df.loc[products_df['Oid']==oid, 'combined_columns'].values[0]
        imgUrl = products_df.loc[products_df['Oid']==oid, 'ImageSource'].values[0].replace('\\','')
        
        print('%s: Final score: %s, Text score: %s, Ordering score: %s, Feedback score: %s \n%s' % (oid, final_score, text_score, 
                                                                                ordering_score, feedback_score, imgUrl))
    
def cv_imshow(image, title=""):
    import cv2
    from matplotlib import pyplot as plt
    if len(np.shape(image))==2:
        plt.title(title)
        plt.imshow(image, cmap='gray')
    if len(np.shape(image))==3 and image.shape[2]==3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        plt.title(title)
        plt.imshow(image)        
    return plt.show()

In [5]:
output = widgets.Output()

search_field = widgets.Text(
    placeholder='Search',
    layout=widgets.Layout(width='30%')
)
numberResults_field = widgets.IntText(
    layout=widgets.Layout(width='5em')
)
search_button = widgets.Button(
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Search',
    icon='fa-search', # (FontAwesome names without the `fa-` prefix)
    layout=widgets.Layout(width='5em')
)
recalc_button = widgets.Button(
    disabled=False,
    button_style='warning', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Recalculate',
    icon='fa-calculator', # (FontAwesome names without the `fa-` prefix)
    layout=widgets.Layout(width='5em')
)

all_widgets = widgets.VBox(
    [widgets.HBox([search_field, numberResults_field, search_button, recalc_button]), 
     output]
)

display(all_widgets)

checkbox_dict = {}
slider_dict = {}

def actionSearch(b):
    with output:
        print('Register new search in Search table')
        searchID = registerSearch(search_field.value, numberResults_field.value, user)
        
        print('Initialize recommender for search Oid: %s' % searchID)
        recommender = FashionRecommender(searchTerm=search_field.value, 
                                         recalc=False, 
                                         user=user,
                                         threshold=threshold)
        print("Executing recommendation.")
        execute_recommendation(recommender)
        
def recalcSearch(b):
    with output:
        print('Get latest search Oid from user %s' % user)
        searchID = db_manager.getLastRecordID(table='Search', where="WHERE CreatedBy=\'%s\'" % user)
        print('Initialize recommender for search Oid: %s' % searchID)
        recommender = FashionRecommender(searchTerm=search_field.value, 
                                         recalc=True, 
                                         user=user,
                                         threshold=threshold)
        #Get user feedback
        do_interaction(recommender)
        print("Executing recommendation correction.")
        execute_recommendation(recommender)

search_button.on_click(actionSearch)
recalc_button.on_click(recalcSearch)

In [6]:
# recommender = FashionRecommender(searchTerm=search_field.value, 
#                                  user=user,
#                                  threshold=threshold)
# recommender.searchID, recommender.searchTerm, recommender.numberResults = recommender.get_search_details()
# recommender.searchID, recommender.searchTerm, recommender.numberResults

In [7]:
# do_interaction(recommender)